ROTATE AND OBSERVATION POINTS RETURN

In [3]:
import numpy as np
import pygame
import math

# Constants
SCREEN_WIDTH = 600
SCREEN_HEIGHT = 600

PREDATOR_RADIUS = 20
FOV_RADIUS = 100

WHITE = (255, 255, 255)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)
YELLOW = (255, 255, 0)
ORANGE = (255, 165, 0)
PURPLE = (128, 0, 128)

pygame.init()
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption('Env render test')

class Agent:
    def __init__(self, agent_name, agent_index):
        self.index = agent_index
        self.agent = agent_name
        self.movement_speed = 1.00
        self.current_position = None
        self.previous_position = np.array([0, 0], dtype=np.float32)
        self.angle = 0
        self.center = 0
        self.direction = 0
        self.direction_end = 0
        self.draw_direction_end = 0
        self.rotation_angle = 20  # Rotation angle when encountering a wall
        self.walls = {
            'wall1': {'x': 150, 'y': 100, 'width': 20, 'height': 100, 'color': BLUE, 'id': 1},
            'wall2': {'x': 250, 'y': 200, 'width': 30, 'height': 60, 'color': GREEN, 'id': 2},
            'wall3': {'x': 350, 'y': 150, 'width': 25, 'height': 30, 'color': YELLOW, 'id': 3},
            'wall4': {'x': 100, 'y': 400, 'width': 20, 'height': 60, 'color': ORANGE, 'id': 4},
            'wall5': {'x': 200, 'y': 300, 'width': 40, 'height': 20, 'color': PURPLE, 'id': 5},
            'wall6': {'x': 450, 'y': 150, 'width': 35, 'height': 45, 'color': RED, 'id': 6},
            'wall7': {'x': 300, 'y': 150, 'width': 25, 'height': 30, 'color': YELLOW, 'id': 7},
            'wall8': {'x': 200, 'y': 400, 'width': 20, 'height': 60, 'color': ORANGE, 'id': 8},
            'wall9': {'x': 700, 'y': 300, 'width': 40, 'height': 20, 'color': PURPLE, 'id': 9},
            'wall10': {'x': 1050, 'y': 150, 'width': 35, 'height': 45, 'color': RED, 'id': 10},
            'wall11': {'x': 450, 'y': 100, 'width': 20, 'height': 100, 'color': BLUE, 'id': 11},
            'wall12': {'x': 950, 'y': 200, 'width': 30, 'height': 60, 'color': GREEN, 'id': 12},
            'wall13': {'x': 150, 'y': 150, 'width': 25, 'height': 30, 'color': YELLOW, 'id': 13},
            'wall14': {'x': 900, 'y': 400, 'width': 20, 'height': 60, 'color': ORANGE, 'id': 14},
            'wall15': {'x': 600, 'y': 300, 'width': 40, 'height': 20, 'color': PURPLE, 'id': 15}
        }

    def agent_reset(self, width, height):
        padding = 30
        self.current_position = np.array(
            [np.random.uniform(30, width - padding), np.random.uniform(30, height - padding)], dtype=np.float32)

        theta = math.radians(self.angle)
        dir_vec_x = padding * math.cos(theta)
        dir_vec_y = padding * math.sin(theta)
        self.direction_end = np.array([self.current_position[0] + dir_vec_x, self.current_position[1] + dir_vec_y],
                                      dtype=np.float32)
        self.draw_direction_end = (self.current_position[0] + dir_vec_x, self.current_position[1] + dir_vec_y)

    def get_direction(self, angle):
        center = (int(self.current_position[0]), int(self.current_position[1]))
        self.center = center

        theta = math.radians(self.angle)
        directional_vector_x = 30 * math.cos(theta)
        directional_vector_y = 30 * math.sin(theta)

        directional_line_end = np.array([center[0] + directional_vector_x, center[1] + directional_vector_y],
                                        dtype=np.float32)
        self.direction_end = directional_line_end

        direction = directional_line_end - center
        direction /= np.linalg.norm(direction)
        self.direction = direction
        self.draw_direction_end = (center[0] + directional_vector_x, center[1] + directional_vector_y)

    def draw_environment(self):
        for wall_data in self.walls.values():
            pygame.draw.rect(screen, wall_data['color'], (wall_data['x'], wall_data['y'], wall_data['width'], wall_data['height']))
            # Draw text label
            font = pygame.font.Font(None, 20)
            label = font.render(f"{wall_data['id']}", True, (255, 255, 255))
            screen.blit(label, (wall_data['x'] + wall_data['width'] // 2 - 10, wall_data['y'] + wall_data['height'] // 2 - 10))

        pygame.draw.circle(screen, BLUE, (int(self.current_position[0]), int(self.current_position[1])), PREDATOR_RADIUS)
        pygame.draw.line(screen, BLUE, (int(self.current_position[0]), int(self.current_position[1])),
                        self.draw_direction_end, 5)
        pygame.draw.circle(screen, BLUE, (int(self.current_position[0]), int(self.current_position[1])), FOV_RADIUS, 1)


    def get_fov_points(self):
        fov_points = {}  

        for x in range(int(self.current_position[0] - FOV_RADIUS), int(self.current_position[0] + FOV_RADIUS)):
            for y in range(int(self.current_position[1] - FOV_RADIUS), int(self.current_position[1] + FOV_RADIUS)):
                if x >= 0 and x < SCREEN_WIDTH and y >= 0 and y < SCREEN_HEIGHT:
                    fov_points[(x, y)] = 0

        for wall_name, wall_data in self.walls.items():
            for x in range(wall_data['x'], wall_data['x'] + wall_data['width']):
                for y in range(wall_data['y'], wall_data['y'] + wall_data['height']):
                    fov_points[(x, y)] = wall_data['color']

        return fov_points

    def step_update(self, action, range_x, range_y, walls):
        self.previous_position = np.copy(self.current_position)

        if action == 0:
            self.angle += 10
            self.get_direction(self.angle)

        elif action == 1:
            self.angle -= 10
            self.get_direction(self.angle)

        elif action == 2:
            new_position = self.current_position + self.direction * self.movement_speed

            if not self.check_wall_collision(new_position, walls):
                self.current_position = new_position
            else:
                self.rotate_towards_open_direction()

            self.get_direction(self.angle)

        elif action == 3:
            new_position = self.current_position - self.direction * self.movement_speed

            if not self.check_wall_collision(new_position, walls):
                self.current_position = new_position
            else:
                self.rotate_towards_open_direction()

            self.get_direction(self.angle)

        elif action == 4:
            pass

        self.current_position[0] = np.clip(self.current_position[0], 0, range_x)
        self.current_position[1] = np.clip(self.current_position[1], 0, range_y)

    

    def check_wall_collision(self, new_position, walls):
        for wall_data in walls.values():
            if (
                new_position[0] + 15 > wall_data['x']
                and new_position[0] - 15 < wall_data['x'] + wall_data['width']
                and new_position[1] + 15 > wall_data['y']
                and new_position[1] - 15 < wall_data['y'] + wall_data['height']
            ):
                return True  # Collision detected

        return False  # No collision

    def rotate_towards_open_direction(self):
        # Rotate either clockwise or anticlockwise based on a random choice
        rotation_direction = np.random.choice([-1, 1])
        self.angle += rotation_direction * self.rotation_angle
        self.get_direction(self.angle)

    def detect_overlapping_walls(self):
        overlapping_walls = []

        for wall_name, wall_data in self.walls.items():
            for x in range(wall_data['x'], wall_data['x'] + wall_data['width']):
                for y in range(wall_data['y'], wall_data['y'] + wall_data['height']):
                    distance_to_wall = np.linalg.norm(np.array([x, y]) - self.current_position)

                    if distance_to_wall <= FOV_RADIUS:
                        overlapping_walls.append({
                            'name': wall_name,
                            'id': wall_data['id'],
                            'color': wall_data['color'],
                            'x': x,
                            'y': y
                        })

        return overlapping_walls


# Create agent and initialize environment
agent = Agent('predator', 1)
agent.agent_reset(SCREEN_WIDTH, SCREEN_HEIGHT)

running = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    screen.fill(WHITE)
    agent.step_update(2, SCREEN_WIDTH, SCREEN_HEIGHT, agent.walls)
    agent.draw_environment()

    overlapping_walls = agent.detect_overlapping_walls()

    # Print details of overlapping walls
    for wall in overlapping_walls:
        print(f"Overlapping Wall - Name: {wall['name']}, ID: {wall['id']}, Coordinates: ({wall['x']}, {wall['y']})")

    pygame.display.update()
    pygame.time.delay(5)

pygame.quit()


Overlapping Wall - Name: wall1, ID: 1, Coordinates: (150, 100)
Overlapping Wall - Name: wall1, ID: 1, Coordinates: (150, 101)
Overlapping Wall - Name: wall1, ID: 1, Coordinates: (150, 102)
Overlapping Wall - Name: wall1, ID: 1, Coordinates: (150, 103)
Overlapping Wall - Name: wall1, ID: 1, Coordinates: (150, 104)
Overlapping Wall - Name: wall1, ID: 1, Coordinates: (150, 105)
Overlapping Wall - Name: wall1, ID: 1, Coordinates: (150, 106)
Overlapping Wall - Name: wall1, ID: 1, Coordinates: (150, 107)
Overlapping Wall - Name: wall1, ID: 1, Coordinates: (150, 108)
Overlapping Wall - Name: wall1, ID: 1, Coordinates: (150, 109)
Overlapping Wall - Name: wall1, ID: 1, Coordinates: (150, 110)
Overlapping Wall - Name: wall1, ID: 1, Coordinates: (150, 111)
Overlapping Wall - Name: wall1, ID: 1, Coordinates: (150, 112)
Overlapping Wall - Name: wall1, ID: 1, Coordinates: (150, 113)
Overlapping Wall - Name: wall1, ID: 1, Coordinates: (150, 114)
Overlapping Wall - Name: wall1, ID: 1, Coordinates: (15

WALLS AND STOP

In [4]:
import pygame
import numpy as np
import math
import random

class GameEnv:
    def __init__(self):
        # Constants
        self.SCREEN_WIDTH = 600
        self.SCREEN_HEIGHT = 600
        self.PREDATOR_RADIUS = 20
        self.FOV_RADIUS = 100
        self.WHITE = (255, 255, 255)
        self.BLUE = (0, 0, 255)

        pygame.init()
        self.screen = pygame.display.set_mode((self.SCREEN_WIDTH, self.SCREEN_HEIGHT))
        pygame.display.set_caption('Simple Env')

        # Random initial position for the predator
        self.predator_position = np.array([random.uniform(30, self.SCREEN_WIDTH - 30), random.uniform(30, self.SCREEN_HEIGHT - 30)],
                                          dtype=np.float32)

        self.predator_angle = 0
        self.predator_speed = 1.0

        self.walls = {
            'wall1': {'x': 150, 'y': 100, 'width': 20, 'height': 100, 'color': BLUE, 'id': 1},
            'wall2': {'x': 250, 'y': 200, 'width': 30, 'height': 60, 'color': GREEN, 'id': 2},
            'wall3': {'x': 350, 'y': 150, 'width': 25, 'height': 30, 'color': YELLOW, 'id': 3},
            'wall4': {'x': 100, 'y': 400, 'width': 20, 'height': 60, 'color': ORANGE, 'id': 4},
            'wall5': {'x': 200, 'y': 300, 'width': 40, 'height': 20, 'color': PURPLE, 'id': 5},
            'wall6': {'x': 450, 'y': 150, 'width': 35, 'height': 45, 'color': RED, 'id': 6},
            'wall7': {'x': 300, 'y': 150, 'width': 25, 'height': 30, 'color': YELLOW, 'id': 7},
            'wall8': {'x': 200, 'y': 400, 'width': 20, 'height': 60, 'color': ORANGE, 'id': 8},
            'wall9': {'x': 700, 'y': 300, 'width': 40, 'height': 20, 'color': PURPLE, 'id': 9},
            'wall10': {'x': 1050, 'y': 150, 'width': 35, 'height': 45, 'color': RED, 'id': 10}
        }

    def draw_environment(self):
        for wall_data in self.walls.values():
            pygame.draw.rect(self.screen, wall_data['color'],
                             (wall_data['x'], wall_data['y'], wall_data['width'], wall_data['height']))

        pygame.draw.circle(self.screen, self.BLUE, (int(self.predator_position[0]), int(self.predator_position[1])),
                           self.PREDATOR_RADIUS)
        pygame.draw.circle(self.screen, self.BLUE, (int(self.predator_position[0]), int(self.predator_position[1])),
                           self.FOV_RADIUS, 1)

    def check_wall_collision(self, new_position):
        for wall_data in self.walls.values():
            if (
                new_position[0] + self.PREDATOR_RADIUS > wall_data['x']
                and new_position[0] - self.PREDATOR_RADIUS < wall_data['x'] + wall_data['width']
                and new_position[1] + self.PREDATOR_RADIUS > wall_data['y']
                and new_position[1] - self.PREDATOR_RADIUS < wall_data['y'] + wall_data['height']
            ):
                return True  # Collision detected
        return False  # No collision

    def update_predator_position(self):
        new_position = self.predator_position + self.predator_speed * np.array(
            [math.cos(math.radians(self.predator_angle)), math.sin(math.radians(self.predator_angle))])

        if not self.check_wall_collision(new_position):
            self.predator_position = new_position

    def run_game(self):
        running = True

        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False

            self.screen.fill(self.WHITE)

            self.update_predator_position()
            self.draw_environment()

            pygame.display.update()
            pygame.time.delay(20)

        pygame.quit()


if __name__ == "__main__":
    game_env = GameEnv()
    game_env.run_game()


In [2]:
pygame.quit()

Slanted walls and stop

In [229]:
import pygame
import numpy as np
import math
import random

class GameEnv:
    def __init__(self):
        # Constants
        self.SCREEN_WIDTH = 600
        self.SCREEN_HEIGHT = 600
        self.PREDATOR_RADIUS = 20
        self.FOV_RADIUS = 100
        self.WHITE = (255, 255, 255)
        self.BLUE = (0, 0, 255)
        self.GREEN = (0, 255, 0)  # Green color for the wall

        pygame.init()
        self.screen = pygame.display.set_mode((self.SCREEN_WIDTH, self.SCREEN_HEIGHT))
        pygame.display.set_caption('Simple Env')

        # Random initial position for the predator
        self.predator_position = np.array([random.uniform(30, self.SCREEN_WIDTH - 30), random.uniform(30, self.SCREEN_HEIGHT - 30)],
                                          dtype=np.float32)

        self.predator_angle = 0
        self.predator_speed = 1.0

        # Slanted wall (diagonally slanted)
        self.slanted_wall_start = (200, 100)
        self.slanted_wall_end = (300, 200)
        self.calculate_slanted_wall()

    def calculate_slanted_wall(self):
        dx = self.slanted_wall_end[0] - self.slanted_wall_start[0]
        dy = self.slanted_wall_end[1] - self.slanted_wall_start[1]
        self.slanted_wall_length = int(np.sqrt(dx**2 + dy**2))

        angle = np.arctan2(dy, dx)
        self.slanted_wall_angle = math.degrees(angle)

        self.slanted_wall_points = np.linspace(self.slanted_wall_start, self.slanted_wall_end, self.slanted_wall_length)

    def draw_environment(self):
        pygame.draw.line(self.screen, self.GREEN, self.slanted_wall_start, self.slanted_wall_end, 45)  # Green wall

        pygame.draw.circle(self.screen, self.BLUE, (int(self.predator_position[0]), int(self.predator_position[1])),
                           self.PREDATOR_RADIUS)
        pygame.draw.circle(self.screen, self.BLUE, (int(self.predator_position[0]), int(self.predator_position[1])),
                           self.FOV_RADIUS, 1)

    def check_wall_collision(self, new_position):
        # Check collision with the slanted wall
        for point in self.slanted_wall_points:
            if (
                new_position[0] + self.PREDATOR_RADIUS > point[0]
                and new_position[0] - self.PREDATOR_RADIUS < point[0]
                and new_position[1] + self.PREDATOR_RADIUS > point[1]
                and new_position[1] - self.PREDATOR_RADIUS < point[1]
            ):
                return True  # Collision detected

        return False  # No collision

    # calculates the new position of the predator based on its current position, 
    # angle of movement (predator_angle), and speed (predator_speed). It uses 
    # trigonometric functions (cos and sin) to determine the change in x and y coordinates 
    # based on the movement angle.
    
    def update_predator_position(self):
        new_position = self.predator_position + self.predator_speed * np.array(
            [math.cos(math.radians(self.predator_angle)), math.sin(math.radians(self.predator_angle))])

        # Checks if the new position would result in a collision with the wall.
        # If there is no collision, the predator is allowed to move.
        if not self.check_wall_collision(new_position):  
            self.predator_position = new_position

    def run_game(self):
        running = True

        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False

            self.screen.fill(self.WHITE)

            self.update_predator_position()
            self.draw_environment()

            pygame.display.update()
            pygame.time.delay(5)

        pygame.quit()


if __name__ == "__main__":
    game_env = GameEnv()
    game_env.run_game()
